# Scratchpad

In [ ]:
# Torch module for learnable rotation matrix
class RotationMatrix(torch.nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim
        # TODO unse only n_angle params
        #num_angles = (dim-1) * dim // 2 
        self.angles = torch.nn.Parameter(torch.zeros(dim, dim))
        self.eye = torch.eye(dim)

    def forward(self, x):
        rotation_matrix = self.eye
        for i in range(self.dim):
            for j in range(i + 1, self.dim):
                vi = self.eye[i]
                vj = self.eye[j]
                V = torch.outer(vi, vi) + torch.outer(vj, vj)
                W = torch.outer(vi, vj) - torch.outer(vj, vi)
                angle = self.angles[i, j]  # TODO unse only n_angle params
                cos_term = torch.cos(angle) - 1
                sin_term = torch.sin(angle)
                rot_ij = self.eye + cos_term * V  + sin_term * W
                rotation_matrix = rot_ij @ rotation_matrix
        return rotation_matrix @ x

In [ ]:
def align(label, pred):
    
    target = pred.detach()
    rot = RotationMatrix(pao_basis_size)
    optim = torch.optim.Adam(rot.parameters())

    for step in range(1000):
        rotated_label = rot(label)
        loss = - (target * rotated_label).sum().pow(2)
        optim.zero_grad()
        loss.backward()
        optim.step()
    #if step % 10 == 0:
    #    print(f"rot  {step:5d} | loss {loss:.8e}")

    angles_sum = rot.angles.abs().sum()
    print(f"aligned, total angle sum: {angles_sum:.1f}, loss: {loss}")
    return rotated_label.detach()

In [ ]:
labels = [align(label, model(edge_vec)) for edge_vec, label in zip(edge_vecs, labels)]